# 0. Anomaly Detection with CapyMOA

This notebook shows some basic usage of CapyMOA for Anomaly Detection tasks.

---

*More information about CapyMOA can be found in* https://www.capymoa.org

**last update on 19/05/2024**

## 1. Unsupervised Anomaly Detection for data streams

* Recent research has been focused on unsupervised anomaly detection for data streams, as it is often difficult to obtain labeled data for training.
* Instead of using evaluation functions such as **test-then-train loop**, we will show a basic loop from scratch to evaluate the model's performance.

In [1]:
from capymoa.datasets import ElectricityTiny
from capymoa.anomaly import HalfSpaceTrees
from capymoa.evaluation import AUCEvaluator
stream = ElectricityTiny()
schema = stream.get_schema()
learner = HalfSpaceTrees(schema)
evaluator = AUCEvaluator(schema)
while stream.has_more_instances():
    instance = stream.next_instance()
    proba = learner.predict_proba(instance)
    evaluator.update(instance.y_index, proba)
    learner.train(instance)
    
auc = evaluator.auc()
print(f"AUC: {auc:.1f}")

AUC: 0.6
